**Install pycaret into environment**

In [1]:
!pip install uv
!uv pip install pycaret[full] --system

Streaming output truncated to the last 5000 lines.
gevent     ------------------------------ 3.16 MB/6.50 MB
aplr       ------------------------------ 3.63 MB/6.61 MB
dash       ------------------------------ 3.58 MB/7.49 MB
⠦ Preparing packages... (87/145)
iterative-telemetry ------------------------------ 10.62 kB/10.62 kB
multimethod ------------------------------ 10.65 kB/10.65 kB
deprecation ------------------------------ 11.18 kB/11.18 kB
colorlog   ------------------------------ 11.36 kB/11.36 kB
flask-wtf  ------------------------------ 12.72 kB/12.72 kB
dacite     ------------------------------ 14.31 kB/14.31 kB
graphql-relay ------------------------------ 16.94 kB/16.94 kB
adagio     ------------------------------ 19.07 kB/19.07 kB
m2cgen     ------------------------------ 64.35 kB/92.24 kB
visions    ------------------------------ 64.43 kB/104.83 kB
litestar   ------------------------------ 335.46 kB/538.37 kB
jedi       ------------------------------ 304.96 kB/1.57 MB
faker

**Mount google drive and copy kaggle.json to machine**

In [2]:
from google.colab import drive
drive.mount('mount')

Mounted at mount


In [3]:
!cp mount/MyDrive/kaggle.json .
!ls

kaggle.json  mount  sample_data


**Acquire the students dataset from kaggle and extract zip**

I am using this dataset uploaded from June 2024 on kaggle:

https://www.kaggle.com/datasets/rabieelkharoua/students-performance-dataset

In [4]:
!KAGGLE_CONFIG_DIR=$(pwd) kaggle datasets download rabieelkharoua/students-performance-dataset
!unzip students-performance-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/rabieelkharoua/students-performance-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
100% 66.2k/66.2k [00:00<00:00, 397kB/s]
100% 66.2k/66.2k [00:00<00:00, 396kB/s]
Archive:  students-performance-dataset.zip
  inflating: Student_performance_data _.csv  


**Rename the csv to remove space in filename**

In [5]:
!ls
!mv 'Student_performance_data _.csv' Student_performance_data_.csv
!head 'Student_performance_data_.csv'

 kaggle.json   sample_data			 students-performance-dataset.zip
 mount	      'Student_performance_data _.csv'
StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
1001,17,1,0,2,19.833722807854713,7,1,2,0,0,1,0,2.929195591667681,2.0
1002,18,0,0,1,15.40875605584674,0,0,1,0,0,0,0,3.042914833436377,1.0
1003,15,0,2,3,4.21056976881226,26,0,2,0,0,0,0,0.1126022544661815,4.0
1004,17,1,0,3,10.028829473958215,14,0,3,1,0,0,0,2.0542181397029484,3.0
1005,17,1,0,2,4.6724952729713305,17,1,3,0,0,0,0,1.2880611817953875,4.0
1006,18,0,0,1,8.191218545250186,0,0,1,1,0,0,0,3.0841836144863937,1.0
1007,15,0,1,1,15.601680474699295,10,0,3,0,1,0,0,2.748237414891583,2.0
1008,15,1,1,4,15.424496305808074,22,1,1,1,0,0,0,1.360142712316461,4.0
1009,17,0,0,0,4.562007558047703,1,0,2,0,1,0,1,2.896819189513569,2.0


**Load the csv into a pandas dataframe**

In [6]:
import pandas as pd
data = pd.read_csv('Student_performance_data_.csv')
data.head()

,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,1003,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,1004,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,1005,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0


In [7]:
data = data.drop(['GradeClass', 'StudentID'], axis=1)
data.head()

,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA
0,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196
1,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915
2,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602
3,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218
4,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061


**Import pycaret and setup a regression experiment**

In [8]:
# import RegressionExperiment and init the class
from pycaret.regression import RegressionExperiment
exp = RegressionExperiment()

# init setup on exp
exp.setup(data, target = 'GPA', session_id = 1337, use_gpu=True)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

,Description,Value
0,Session id,1337
1,Target,GPA
2,Target type,Regression
3,Original data shape,"(2392, 13)"
4,Transformed data shape,"(2392, 13)"
5,Transformed train set shape,"(1674, 13)"
6,Transformed test set shape,"(718, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

In [9]:
exp.get_config()

{'USI',
 'X',
 'X_test',
 'X_test_transformed',
 'X_train',
 'X_train_transformed',
 'X_transformed',
 '_available_plots',
 '_ml_usecase',
 'data',
 'dataset',
 'dataset_transformed',
 'exp_id',
 'exp_name_log',
 'fold_generator',
 'fold_groups_param',
 'fold_shuffle_param',
 'gpu_n_jobs_param',
 'gpu_param',
 'html_param',
 'idx',
 'is_multiclass',
 'log_plots_param',
 'logging_param',
 'memory',
 'n_jobs_param',
 'pipeline',
 'seed',
 'target_param',
 'test',
 'test_transformed',
 'train',
 'train_transformed',
 'transform_target_param',
 'variable_and_property_keys',
 'variables',
 'y',
 'y_test',
 'y_test_transformed',
 'y_train',
 'y_train_transformed',
 'y_transformed'}

In [10]:
exp.get_config('train_transformed').head()

,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA
1229,17.0,1.0,0.0,2.0,4.185755,0.0,0.0,2.0,1.0,1.0,0.0,0.0,3.343051
1705,18.0,1.0,0.0,2.0,8.858282,0.0,1.0,4.0,1.0,0.0,0.0,0.0,4.000000
2285,18.0,0.0,3.0,1.0,0.393664,14.0,1.0,2.0,0.0,1.0,1.0,1.0,2.095648
2025,16.0,1.0,0.0,1.0,8.858743,0.0,1.0,2.0,0.0,1.0,1.0,0.0,3.748102
2106,17.0,0.0,0.0,2.0,14.430236,26.0,1.0,4.0,0.0,0.0,0.0,0.0,0.989468


In [11]:
exp.get_config('test_transformed').head()

,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA
571,17.0,1.0,3.0,2.0,14.832756,12.0,0.0,3.0,0.0,0.0,0.0,0.0,2.119171
157,16.0,0.0,0.0,2.0,10.369860,6.0,0.0,2.0,0.0,0.0,0.0,0.0,2.784238
1680,18.0,0.0,1.0,3.0,4.285436,29.0,1.0,2.0,0.0,1.0,0.0,0.0,0.397989
73,16.0,0.0,0.0,3.0,16.248274,4.0,1.0,2.0,0.0,0.0,0.0,0.0,3.300929
2016,15.0,1.0,0.0,0.0,18.677176,27.0,1.0,4.0,0.0,0.0,0.0,0.0,1.139337


In [12]:
print("Data length: ", len(data))
print("Train Data length: ", len(exp.get_config('train_transformed')))
print("Test Data length: ", len(exp.get_config('test_transformed')))
print()

if len(data) == len(exp.get_config('train_transformed')) + len(exp.get_config('test_transformed')):
  print("Lengths match as expected")

Data length:  2392
Train Data length:  1674
Test Data length:  718

Lengths match as expected


**Train models**

In [13]:
best = exp.compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.1575,0.0384,0.1957,0.9535,0.0803,0.1543,0.0280
ridge,Ridge Regression,0.1575,0.0384,0.1957,0.9535,0.0803,0.1544,0.0250
lar,Least Angle Regression,0.1575,0.0384,0.1957,0.9535,0.0803,0.1543,0.0250
br,Bayesian Ridge,0.1575,0.0384,0.1957,0.9535,0.0803,0.1545,0.0290
huber,Huber Regressor,0.1583,0.0387,0.1963,0.9531,0.0807,0.1565,0.2010
catboost,CatBoost Regressor,0.1621,0.0410,0.2021,0.9504,0.0818,0.1568,8.2850
gbr,Gradient Boosting Regressor,0.1704,0.0453,0.2125,0.9451,0.0864,0.1689,0.2690
lightgbm,Light Gradient Boosting Machine,0.1791,0.0508,0.2247,0.9385,0.0906,0.1732,0.0650
xgboost,Extreme Gradient Boosting,0.1931,0.0591,0.2426,0.9284,0.0976,0.1877,0.2610
et,Extra Trees Regressor,0.1942,0.0601,0.2447,0.9270,0.0979,0.1854,0.4300


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

**Evaluate Model**

In [14]:
exp.evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

**Predict on test data using Model**

In [15]:
predict_holdout = exp.predict_model(best)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Linear Regression,0.1634,0.0403,0.2006,0.9514,0.0837,0.1566


In [16]:
new_data = data.copy()
new_data = new_data.drop('GPA', axis=1)

In [17]:
predictions = exp.predict_model(best, data=new_data)
predictions.head(10)

,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,prediction_label
0,17,1,0,2,19.833723,7,1,2,0,0,1,0,3.084253
1,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.095597
2,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.359524
3,17,1,0,3,10.028830,14,0,3,1,0,0,0,2.059484
4,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.661693
5,18,0,0,1,8.191218,0,0,1,1,0,0,0,3.082144
6,15,0,1,1,15.601681,10,0,3,0,1,0,0,2.620562
7,15,1,1,4,15.424497,22,1,1,1,0,0,0,1.387279
8,17,0,0,0,4.562007,1,0,2,0,1,0,1,3.025813
9,16,1,0,1,18.444466,0,0,3,1,0,0,0,3.699196


**Save the model to disk**

In [18]:
# save pipeline
exp.save_model(best, 'best_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Age', 'Gender', 'Ethnicity',
                                              'ParentalEducation',
                                              'StudyTimeWeekly', 'Absences',
                                              'Tutoring', 'ParentalSupport',
                                              'Extracurricular', 'Sports',
                                              'Music', 'Volunteering'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('trained_model', LinearRegression(n_jobs=-1))]),
 'best_model.pkl')

**Further tune the model**

In [19]:
tuned_best_model = exp.tune_model(best)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.1366,0.0307,0.1751,0.9678,0.0675,0.1008
1,0.1676,0.0429,0.2070,0.9478,0.0863,0.1514
2,0.1706,0.0441,0.2100,0.9482,0.0878,0.1745
3,0.1488,0.0361,0.1899,0.9563,0.0773,0.1051
4,0.1431,0.0315,0.1776,0.9644,0.0720,0.1981
5,0.1570,0.0379,0.1948,0.9481,0.0804,0.1060
6,0.1636,0.0407,0.2016,0.9496,0.0829,0.1698
7,0.1609,0.0383,0.1956,0.9575,0.0843,0.1627
8,0.1714,0.0448,0.2116,0.9453,0.0810,0.1190


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 2 candidates, totalling 20 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [20]:
exp.save_model(tuned_best_model, 'tuned_best_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Age', 'Gender', 'Ethnicity',
                                              'ParentalEducation',
                                              'StudyTimeWeekly', 'Absences',
                                              'Tutoring', 'ParentalSupport',
                                              'Extracurricular', 'Sports',
                                              'Music', 'Volunteering'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('trained_model', LinearRegression(n_jobs=-1))]),
 'tuned_best_model.pkl')

**Explainer Dashboard**

In [21]:
exp.dashboard(tuned_best_model, display_format ='inline')

Generating self.shap_explainer = shap.LinearExplainer(modelX)...
Building ExplainerDashboard..
For this type of model and model_output interactions don't work, so setting shap_interaction=False...
The explainer object has no decision_trees property. so setting decision_trees=False...
Generating layout...
Calculating shap values...
Calculating predictions...
Calculating residuals...
Calculating absolute residuals...
Calculating dependencies...
Calculating importances...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Starting ExplainerDashboard inline (terminate it with ExplainerDashboard.terminate(8050))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Generate gradio application**

In [22]:
exp.create_app(tuned_best_model, app_kwargs={})

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c17932809d8e2bf032.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


**Finalize best model**

In [23]:
final_best = exp.finalize_model(tuned_best_model)
exp.save_model(final_best, 'final_best')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Age', 'Gender', 'Ethnicity',
                                              'ParentalEducation',
                                              'StudyTimeWeekly', 'Absences',
                                              'Tutoring', 'ParentalSupport',
                                              'Extracurricular', 'Sports',
                                              'Music', 'Volunteering'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('actual_estimator', LinearRegression(n_jobs=-1))]),
 'final_best.pkl')